# Data Aggregation and Group Operations

_split-apply-combine_

* [1 GroupBy](#1)
* [2 Data aggregation](#2)
* [3 Apply](#3)
* [4 Pivot tables](#4)

## 1 GroupBy <a class="anchor" id="1"></a>

In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),

                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.531384,0.625509
1,a,two,1.480337,-1.567477
2,b,one,0.949595,0.174690
3,b,two,1.769191,-0.586094
4,a,one,-0.100689,0.989657


#### Using a DataFrame

Compute mean of `data1` using the labels from `key1`

In [11]:
## The .groupby() way:
grouped = df['data1'].groupby(df['key1']) #Access data1, call .groupby() using labels from key1
# this generates a GroupBy object
print(type(grouped))
# call .mean() on the grouped object
grouped.mean()

<class 'pandas.core.groupby.groupby.SeriesGroupBy'>


key1
a    0.282755
b    1.359393
Name: data1, dtype: float64

In [16]:
# Multiple columns can be used to group
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means #.reset_index() #if used, the indices will be columns instead of well...indices

key1  key2
a     one    -0.316036
      two     1.480337
b     one     0.949595
      two     1.769191
Name: data1, dtype: float64

In [17]:
# wide format
means.unstack()

key2,one,two
key1,,
a,-0.316036,1.480337
b,0.949595,1.769191


#### Series 

When the key values are not part of the data  

In [19]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

print(states)
print(years)
print(type(states))

['Ohio' 'California' 'California' 'Ohio' 'Ohio']
[2005 2005 2006 2005 2006]
<class 'numpy.ndarray'>


The `states` and `years` are the same size as the rows in the `df` object

In [24]:
len(df) == len(states) == len(years)

True

In [20]:
df['data1'].groupby([states, years]).mean()

California  2005    1.480337
            2006    0.949595
Ohio        2005    0.618904
            2006   -0.100689
Name: data1, dtype: float64

#### Nuissance columns

When aggregating by groups, all numeric columns are used (unless filtered).

In [26]:
# the key2 column is not in the output because it's not a numeric column 
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.282755,0.015896
b,1.359393,-0.205702


#### Size

To count the size of groups

In [27]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

## Iterating Over Groups


The GroupBy object supports iteration, generating a sequence of 2-tuples containing the group name along with the chunk of data.

In [30]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.531384  0.625509
1    a  two  1.480337 -1.567477
4    a  one -0.100689  0.989657
b
  key1 key2     data1     data2
2    b  one  0.949595  0.174690
3    b  two  1.769191 -0.586094


In the case of multiple keys, the __first element__ in the tuple will be a tuple of key values

In [31]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.531384  0.625509
4    a  one -0.100689  0.989657
('a', 'two')
  key1 key2     data1     data2
1    a  two  1.480337 -1.567477
('b', 'one')
  key1 key2     data1    data2
2    b  one  0.949595  0.17469
('b', 'two')
  key1 key2     data1     data2
3    b  two  1.769191 -0.586094


This is useful to computing a `dict` of the data pieces as a one-liner


In [35]:
pieces = dict(list(df.groupby('key1')))
pieces["b"]

,key1,key2,data1,data2
2,b,one,0.949595,0.174690
3,b,two,1.769191,-0.586094


Using multiple keys causes the keys of the dict to be tuples:

In [37]:
pieces_double = dict(list(df.groupby(['key1', 'key2'])))
pieces_double.keys()

dict_keys([('a', 'one'), ('a', 'two'), ('b', 'one'), ('b', 'two')])

In [38]:
pieces_double["a","one"]

,key1,key2,data1,data2
0,a,one,-0.531384,0.625509
4,a,one,-0.100689,0.989657


#### Axis

.groupby() by default works on the `axis=0` (rows). Using `axis=1` groups the columns

In [43]:
df.dtypes # Gets the object type of each column

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [46]:
grouped = df.groupby(df.dtypes, axis=1)
# Groups the columns by dtype (Data type) 
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.531384  0.625509
1  1.480337 -1.567477
2  0.949595  0.174690
3  1.769191 -0.586094
4 -0.100689  0.989657
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


Types of dtypes: https://pbpython.com/pandas_dtypes.html

#### Selecting a Column or Subset of Columns

Indexing a GroupBy object created from a DataFrame with a column name (or array of column names) has the effect of column subsetting for aggregation.

In [63]:
df.groupby('key1')['data1'] #is the same as 
df['data1'].groupby(df['key1'])

In [65]:
df.groupby('key1')[['data2']] #aka
df[['data2']].groupby(df['key1'])

__Two ways:__

In [81]:
## This has the effect of aggregating only by select columns:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.807583
     two  -1.567477
b    one   0.174690
     two  -0.586094

In [82]:
df['data2'].groupby([df['key1'] , df['key2']]).mean()

key1  key2
a     one     0.807583
      two    -1.567477
b     one     0.174690
      two    -0.586094
Name: data2, dtype: float64

In [88]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
print(type(s_grouped)) #grouped Series if only one column name is passed
print()
#####
    type(df.groupby(['key1', 'key2'])['data2','data1']) #or a grouped DataFrame if more than one columns are passed

<class 'pandas.core.groupby.groupby.SeriesGroupBy'>



pandas.core.groupby.groupby.DataFrameGroupBy

### Grouping with Dicts and Series

Grouping information may exist in a form other than an array.

In [91]:
## create a new data frame
people = pd.DataFrame(np.random.randn(5, 5),
                              columns=['a', 'b', 'c', 'd', 'e'],
                              index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
# add NA
print(people.iloc[2:3, [1, 2]])
people.iloc[2:3, [1, 2]] = np.nan
people

            b         c
Wes  0.192059 -0.576063


,a,b,c,d,e
Joe,-0.499377,-0.242713,0.626838,1.810732,1.790153
Steve,1.555727,0.466610,-0.392568,-0.759842,-1.915612
Wes,0.622234,NaN,NaN,-0.552912,-0.545523
Jim,0.130397,0.730735,1.110486,0.739214,0.743815
Travis,0.504124,-0.279970,-0.396885,0.270591,-0.857012


__group correspondence__

We can create a `dict` or `Series` object that maps each column to a group. We can pass the object directly to .groupby(). If values from the dictionary are missing in the data frame they'll just be ignored.

In [94]:
### We can use a dictionary to map column names to groups
mapping = {'a': 'red', 
           'b': 'red', 
           'c': 'blue',
           'd': 'blue', 
           'e': 'red', 
           'f' : 'orange'} # the f column doesn't appear in our table

In [96]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,2.437570,1.048064
Steve,-1.152410,0.106725
Wes,-0.552912,0.076711
Jim,1.849701,1.604946
Travis,-0.126293,-0.632858


In [97]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [100]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


This is counting the number of cells for each person that have a number. `a b` and `e` belong to the red group. `c` and `d` belong to the blue group. 

Since most of the table is filled, blue will have 2 counts per person while red will have 3.

__NA__ <br>
Wes has only 1 count since the values for `b` and `c` columns are NAs. 

### Grouping with functions

Any function passed as a group key will be called once per index value, with the return values being used as the group names.

In [103]:
people.groupby(len).sum()

,a,b,c,d,e
3,0.253254,0.488022,1.737325,1.997034,1.988445
5,1.555727,0.466610,-0.392568,-0.759842,-1.915612
6,0.504124,-0.279970,-0.396885,0.270591,-0.857012


In [108]:
# The function len is passed to the indices. 
# Returns the length of each value in the index.
[len(peps) for peps in people.index]

[3, 5, 3, 3, 6]

Mixing functions with arrays, dicts, or Series is not a problem as everything gets converted to arrays internally

In [109]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.499377 -0.242713  0.626838 -0.552912 -0.545523
  two  0.130397  0.730735  1.110486  0.739214  0.743815
5 one  1.555727  0.466610 -0.392568 -0.759842 -1.915612
6 two  0.504124 -0.279970 -0.396885  0.270591 -0.857012

### Grouping by Index Levels

For hierarchically indexed datasets: ability to aggregate using one of the levels of an axis index.

In [111]:
# Generate multiple indices to be used as hierarchical index
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                            [1, 3, 5, 1, 3]],
                                            names=['cty', 'tenor'])
columns

MultiIndex(levels=[['JP', 'US'], [1, 3, 5]],
           labels=[[1, 1, 1, 0, 0], [0, 1, 2, 0, 1]],
           names=['cty', 'tenor'])

In [112]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      1.078591 -0.298519 -0.497413  0.968533 -0.974431
1     -0.178842 -0.486060  2.233852  0.665555  0.188100
2      0.819590 -1.202499 -0.820831 -0.790126  0.280787
3      2.439811  1.666996  0.244209  0.244187 -0.699329

In [113]:
# To group by level, pass the level number or name using the level keyword:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 2 Data Aggregation <a class="anchor" id="2"></a>

Any transformation that produces scalar.

You can use aggregations of your own devising and additionally call any method that is also defined on the grouped object.

`GroupBy` slices all the Series, calls the method for each one and reassembles everything into the resulting object:

In [118]:
grouped = df.groupby('key1')

In [119]:
# Quantile
grouped['data1'].quantile(0.9)

key1
a    1.164132
b    1.687231
Name: data1, dtype: float64

In [121]:
# A summary of the gorups
grouped['data1'].describe()

,count,mean,std,min,25%,50%,75%,max
key1,,,,,,,,
a,3.0,0.282755,1.059258,-0.531384,-0.316036,-0.100689,0.689824,1.480337
b,2.0,1.359393,0.579542,0.949595,1.154494,1.359393,1.564292,1.769191


#### Custom agg functions
Custom aggregation functions are generally much slower than the optimized functions:


* `count`
* `sum`
* `mean`
* `median`
* `std`, `var` 
* `min`, `max`
* `prod`
* `first`, `last`


In [25]:
# Define a function that aggregates (transforms) an array 
# and pass it to the .agg() method
def peak_to_peak(arr):
    return arr.max() - arr.min()


In [127]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,2.011721,2.557134
b,0.819596,0.760784


In [128]:
# methods like describe also work. 
# (even though they are not aggregations, strictly speaking)
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.282755  1.059258 -0.531384 -0.316036 -0.100689  0.689824   
b      2.0  1.359393  0.579542  0.949595  1.154494  1.359393  1.564292   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.480337   3.0  0.015896  1.383277 -1.567477 -0.470984  0.625509   
b     1.769191   2.0 -0.205702  0.537956 -0.586094 -0.395898 -0.205702   

                          
           75%       max  
key1                      
a     0.807583  0.989657  
b    -0.015506  0.174690

#### Column-Wise and Multiple Function Application

In [5]:
tips = pd.read_csv('examples/tips.csv')
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [6]:
# Add a new column with the proportion of the tip
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


Aggregate using a different function depending on the column, or multiple functions at once. 

In [7]:
# Group the data by day and smoker columns
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct'] #Subset the tip proportion column

__Series__

In [8]:
# The function can be passed as a string to the agg method
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [9]:
grouped_pct.mean()

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [10]:
# Passing a list of functions gets back a DataFrame 
grouped_pct.agg(['mean', 'std', peak_to_peak])

NameError: name 'peak_to_peak' is not defined

Passing a list of (name, function) tuples, the first element of each tuple will be used as the DataFrame column names.

In [11]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

__DataFrame__

With a DataFrame you have more options, as you can specify a list of functions to apply to all of the columns or different functions per column. 

In [12]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [13]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [14]:
# Same as before, a list of tuples (name, function) can be passed
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

Using a `dict`, functions can be applied to different columns 

In [15]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [16]:
# The resulting object (DataFrame) has hierarchical columns 
# only if multiple functions are applied to at least one column
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

#### Returning Aggregated Data Without Row Indexes

Pass `as_index=False` to groupby

In [17]:
# as_index=False method avoids some unnecessary computations
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [18]:
#.reset_index() also works
tips.groupby(['day', 'smoker']).mean().reset_index()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## 3 Apply <a class="anchor" id="3"></a>

`apply` splits the object being manipulated into pieces, invokes the passed function on each piece, and then attempts to concate‐ nate the pieces together.

In [19]:
# Select the top 5 values by group of a certain column:

# Write a function that selects the column, sorts and picks the largest top n
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [20]:
top(tips,n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


Using groupby we can select a column with keys and use it apply our function to the groups. 

In [21]:
#The function is called on each row group of the DataFrame, then put back together using 
# pandas.concat
# Labels are the group names
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [22]:
# it can be applied to more columns
tips.groupby(['smoker','day']).apply(top).head()

total_bill   tip smoker  day    time  size   tip_pct
smoker day                                                          
No     Fri 99        12.46  1.50     No  Fri  Dinner     2  0.120385
           94        22.75  3.25     No  Fri  Dinner     2  0.142857
           91        22.49  3.50     No  Fri  Dinner     2  0.155625
           223       15.98  3.00     No  Fri   Lunch     3  0.187735
       Sat 228       13.28  2.72     No  Sat  Dinner     2  0.204819

In [23]:
# Arguments to the function can be passed like so:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [24]:
# Arguments to the function can be passed like so:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

#### Suppressing the Group Keys

Using `group_keys=False` in the `groupby` call won't use the key as an index.

In [25]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


#### Quantile and Bucket Analysis

Combining `qcut` and `cut` with groupby makes it convenient to perform bucket or quantile analysis on a dataset. 

In [26]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0      (-0.16, 1.456]
1      (-0.16, 1.456]
2    (-3.398, -1.776]
3      (1.456, 3.072]
4     (-1.776, -0.16]
5      (-0.16, 1.456]
6      (-0.16, 1.456]
7     (-1.776, -0.16]
8     (-1.776, -0.16]
9      (-0.16, 1.456]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.398, -1.776] < (-1.776, -0.16] < (-0.16, 1.456] < (1.456, 3.072]]

In [30]:
## The object returned by cut can be passed directly to groupby

def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}

In [33]:
## Group by quartiles
grouped = frame.data2.groupby(quartiles)
## Apply functions to each quartile
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.398, -1.776]",56.0,1.604815,-0.044088,-1.835902
"(-1.776, -0.16]",400.0,2.875072,-0.028024,-2.652749
"(-0.16, 1.456]",483.0,3.174286,0.061941,-3.590257
"(1.456, 3.072]",61.0,2.288842,0.055965,-1.718713


These were equal-length buckets; to compute equal-size buckets based on sample quantiles, use `qcut`.

In [52]:
#Use labels=False to just get quantile numbers

grouping = pd.qcut(frame.data1, 10, labels=False) 
#Groups data into 10 equal groups:
print(grouping.iloc[:].unique() )
# Each value in the data is assigned to a group based on its quantile:
print(grouping.head()) ; print(frame.data1.head(5))

[7 0 9 3 5 2 8 1 6 4]
0    7
1    7
2    0
3    9
4    3
Name: data1, dtype: int64
0    0.584245
1    0.587594
2   -2.758452
3    2.134254
4   -0.378358
Name: data1, dtype: float64


In [53]:
# Group data
grouped = frame.data2.groupby(grouping)

In [54]:
# Apply function(s)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,1.604815,-0.115818,-2.205939
1,100.0,2.820176,0.091459,-2.652749
2,100.0,2.376853,-0.050633,-2.262582
3,100.0,2.875072,-0.112257,-2.485545
4,100.0,2.020544,-0.005697,-1.943735
5,100.0,2.665001,0.179417,-3.590257
6,100.0,2.427609,0.022980,-2.800978
7,100.0,3.174286,0.088956,-2.401369
8,100.0,3.014527,0.050355,-2.663824


#### Example: Filling Missing Values with Group-Specific Values

In [57]:
#Generate 6 random numbers and make some of them NA
s = pd.Series(np.random.randn(6))
s
s[::2] = np.nan
s

0         NaN
1    1.092887
2         NaN
3   -1.580288
4         NaN
5   -0.961824
dtype: float64

In [59]:
# We could substitute them by the mean value of the data
print(s.mean())
s.fillna(s.mean())

-0.4830753772639582


0   -0.483075
1    1.092887
2   -0.483075
3   -1.580288
4   -0.483075
5   -0.961824
dtype: float64

__Fill NA values by groups__:

In [60]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -1.699109
New York     -0.171265
Vermont      -0.074031
Florida      -0.198632
Oregon        0.324820
Nevada       -0.591596
California   -0.209599
Idaho        -1.939989
dtype: float64

In [62]:
# Make some data NA
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -1.699109
New York     -0.171265
Vermont            NaN
Florida      -0.198632
Oregon        0.324820
Nevada             NaN
California   -0.209599
Idaho              NaN
dtype: float64

In [63]:
# Just as before, we can get the mean of the groups using:
data.groupby(group_key).mean()


East   -0.689669
West    0.057610
dtype: float64

In [64]:
## To fill the NA values by group 
fill_mean = lambda g: g.fillna(g.mean()) #Use a lambda function that uses fillna with the mean of the group.


In [69]:
# Group the data by a key, and call the lambda function
print(data)
print("\n////\n")
data.groupby(group_key).apply(fill_mean)

Ohio         -1.699109
New York     -0.171265
Vermont            NaN
Florida      -0.198632
Oregon        0.324820
Nevada             NaN
California   -0.209599
Idaho              NaN
dtype: float64

////



Ohio         -1.699109
New York     -0.171265
Vermont      -0.689669
Florida      -0.198632
Oregon        0.324820
Nevada        0.057610
California   -0.209599
Idaho         0.057610
dtype: float64

Using a `dict` we can substitute or fill NA fields using predetermined values

In [72]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio         -1.699109
New York     -0.171265
Vermont       0.500000
Florida      -0.198632
Oregon        0.324820
Nevada       -1.000000
California   -0.209599
Idaho        -1.000000
dtype: float64

#### Example: Random Sampling and Permutation

Draw a random sample (with/without replacement) from a large dataset. 

Perform the “draws” using the `.sample` method for Series.



In [86]:
# Build a deck:
suits = ['H', 'S', 'C', 'D'] #Four types of cards
card_val = (list(range(1, 11)) + [10] * 3) * 4  #Assign a value to each card 
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q'] 

cards = [] # Empty list
# Fill in the list
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

In [87]:
cards[3:13]

['4H', '5H', '6H', '7H', '8H', '9H', '10H', 'JH', 'KH', 'QH']

### !
<span style="background-color: #FFFF00">We can rewrite this using a __nested__ list comprehension</span>

In [94]:
cardsLC=[str(num)+suit for suit in suits for num in base_names ]
print(type(cardsLC))
cardsLC[3:13]

<class 'list'>


['4H', '5H', '6H', '7H', '8H', '9H', '10H', 'JH', 'KH', 'QH']

In [96]:
cards == cardsLC

True

In [101]:
## Asign values to the cards
deck = pd.Series(card_val, index=cards)
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [102]:
# write a function to draw five cards
def draw(deck, n=5):
    return deck.sample(n)

In [111]:
# write a lambda function that looks into the suit 
# the suit is the second element in the card's index  
get_suit = lambda card: card[-1]

In [115]:
## Group by suit and draw two cards of each suit
deck.groupby(get_suit).apply(draw, n=2)

C  6C     6
   5C     5
D  8D     8
   4D     4
H  QH    10
   5H     5
S  3S     3
   AS     1
dtype: int64

In [119]:
# to remove the hierarchical index
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

5C      5
2C      2
5D      5
10D    10
9H      9
4H      4
9S      9
QS     10
dtype: int64

#### Example: Group Weighted Average and Correlation

Operations between columns in a DataFrame or two Series, such as a group weighted average, are possible.

In [123]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
print(df)

  category      data   weights
0        a -0.701010  0.029804
1        a -0.432810  0.477445
2        a  0.791842  0.938515
3        a  0.209862  0.828972
4        b -1.807315  0.805754
5        b -0.622307  0.099841
6        b  0.141823  0.894519
7        b  0.362541  0.228918


In [126]:
#Group by category
grouped = df.groupby('category')

In [127]:
# Define a function to get the wheighted average using numpy
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

In [128]:
# Apply the weighted average function to each group
grouped.apply(get_wavg)

category
a    0.303151
b   -0.644902
dtype: float64

In [129]:
grouped.apply(np.average)

category
a    0.267828
b    0.012972
dtype: float64

In [132]:
# Contrast with the unweighted average
grouped.apply(np.average)

category
a    0.267828
b    0.012972
dtype: float64

---

Consider a __financial dataset__

It contains end-of-day prices for a few stocks and the S&P 500 index

In [144]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True,index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


Compute the yearly correlations of daily returns (computed from percent changes) with SPX. 

1. First create a function that computes the pairwise correlation of each column with the 'SPX' column.
2. Compute percent change on close_px using pct_change.
3. Group these percent changes by year 

In [146]:
# 1
spx_corr = lambda x: x.corrwith(x['SPX'])
# 2
rets = close_px.pct_change().dropna()
# 3
get_year = lambda x: x.year # lambda function to extract the year from the year attribute
by_year = rets.groupby(get_year) # Group by year
by_year.apply(spx_corr) #Apply to each of the columns: correlate with SPX

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [147]:
# Intercolumn correlations can be obtained as:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

#### Example: Group-Wise Linear Regression

Use groupby to perform more complex group-wise statistical analysis (as long as the function returns a pandas object or scalar value). 

In [152]:
import statsmodels.api as sm 
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars] 
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params


In [153]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 10.4 Pivot Tables & Cross-Tabulation <a class="anchor" id="4"></a>

A pivot table is a data summarization tool frequently found in spreadsheet programs and other data analysis software.

> `It aggregates a table of data by one or more keys, arranging the data in a rectangle with some of the group keys along the rows and some along the columns.`


`pivot_table` can add partial totals, also known as margins.

In [154]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [156]:
# default pivot table calculates mean
# of each group
# Note columns are in different order
tips.groupby(['day', 'smoker']).mean()

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [158]:
# aggregate only tip_pct and size, and additionally group by time.
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [159]:
# We could augment this table to include partial totals by passing margins=True. 
# This has the effect of adding all row and column labels, 
# with corresponding values being the group statistics for all the data within a single tier:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

the __All__ values are means without taking into account __smoker__ versus __non-smoker__ (the All columns) or any of the two levels of grouping on the rows (the All row).

In [160]:
# To use a different aggregation function, pass it to aggfunc. 
# For example, 'count' or len will give you a cross-tabulation (count or frequency) of group sizes:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
                 aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [161]:
## If some combinations are empty (or otherwise NA), you may wish to pass a fill_value:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

#### Cross-tabulation `crosstab`

Special case of a pivot table: computes group frequencies. 

In [164]:
data #In the book the data contains samples from different nationalities and if it's right or left-handed...
# also, it comes out of nowhere...

Ohio         -1.699109
New York     -0.171265
Vermont            NaN
Florida      -0.198632
Oregon        0.324820
Nevada             NaN
California   -0.209599
Idaho              NaN
dtype: float64

In [165]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244